In [1]:
import pandas as pd
import numpy as np
import datetime
from ics import Calendar, Event
import boto3

In [22]:
df_events = pd.read_csv('df_event.csv')

In [23]:
df_event_activity = df_events.query('event_type in ("mindful", "exercise", "activity")').copy().sort_values(by= ['event_type'], ascending= False)
df_event_activity.fillna('', inplace=True)
df_event_activity['description'] = df_event_activity.groupby('date')['description'].transform(lambda x: ' '.join(x))
df_event_activity['description'] = [x.strip() for x in df_event_activity['description']]
df_event_activity['description'] = [x.replace(" ", "") for x in df_event_activity['description']]
df_event_activity.drop_duplicates(subset = ['date', 'description'], keep = 'last')


In [2]:
def ts_to_dt(ts):
    return datetime.datetime.fromtimestamp(ts)

In [3]:
def process_raw_data(file):
    """
    Create [date, source] from file.
    :param file: as exported by Auto Health Export
    """
    df = pd.read_csv(file, sep = ',')
    df['creation_date'] = ts_to_dt(file.stat().st_atime)
    df['filename'] = file.name

    return df

In [189]:
def read_raw_files(str_path):
    """
    Read all files in a directory and return a dataframe.
    :param str_path: directory path as type string
    """
    df = pd.DataFrame()
    print('Reading files..')
    for i in os.scandir(str_path):
        if i.name.startswith('HealthAutoExport') and i.name.endswith('Data.csv'):
            print(f'Reading: {i.name}')
            df_tmp = process_raw_data(i)
            df = pd.concat([df, df_tmp])

    # concat results in weird indices
    df = df.reset_index(drop=True)
    return df 

### Transformations

Functions to cleanse the data 
- Dedupe values
- Cleanse trim all values to closest integer except for sleep and weight
- Create the following columns
  - `Calories`

In [491]:
def create_numeric_cols(df):
    """
    Calculates the total calories from the macros
    Calculates the sleep efficiency
    """
    df['calories'] = df['carbs'] * 4 + df['fat'] * 9 + df['protein'] * 4
    df['sleep_eff'] = df['sleep_asleep'] / df['sleep_in_bed'] * 100
    df['sleep_eff'] = df['sleep_eff'].fillna(0)
    df['sleep_eff'] = df['sleep_eff'].astype(int)

    return df

In [492]:
def round_df(df):
    """
    Round all numerical columns to closest integer except for one d.p. cols 
    Replaces all NaN with null
    """
    one_dp_cols = ['sleep_asleep', 'sleep_in_bed', 'weight']
    for i in df.columns:
        if df[i].dtypes == 'float64':
            if i in one_dp_cols: 
                df[i] = df[i].round(1)
            else:
                df[i] = df[i].astype(int)

    df = df.replace({np.nan: None})
    return df

In [493]:
def convert_column_types(df): 
    """
    Convert certain columns to be a certain type 
    """
    df['date'] = pd.to_datetime(df['date']).dt.date

    # force apply float64 type for weight 
    df['weight'] = df['weight'].astype(float)

    return df

In [494]:
def rename_columns(df):
    """
    Rename columns for easier reference
    Styling follows lowercase and no units with spaces being replaced by _
    """
    d_col_rename = {
        'Date': 'date',
        'Carbohydrates (g)': 'carbs',
        'Protein (g)': 'protein',
        'Total Fat (g)': 'fat',
        'Sleep Analysis [In Bed] (hr)': 'sleep_in_bed',
        'Sleep Analysis [Asleep] (hr)': 'sleep_asleep',
        'Step Count (count)': 'steps',
        'Weight & Body Mass (kg) ': 'weight'
    }

    df.rename(columns=d_col_rename, inplace=True)

    # fill in values 
    df = df.replace(r'^\s+$', np.nan, regex=True)

    # convert column types 
    df = convert_column_types(df)
    return df

In [495]:
def dedup_df(df):
    """
    Remove duplicates ordering by 'date' and 'creation_date' and then keep only the latest 
    """
    df_sort = df.sort_values(['date', 'creation_date'], ascending= True)
    df_dedup = df_sort.drop_duplicates(subset = 'date', keep = 'last')

    return df_dedup

In [506]:
def create_description_cols(df): 
    """
    Create description columns for the generating events
    """
    # convert all columns to strings for easy manipulation
    df_1 = df.astype(str)

    food_macros = "(" + df_1['carbs'] + "C/" + df_1['protein'] + "P/" + df_1['fat'] + "F" + ")"
    df['food'] = df_1['calories'] + " calories " + food_macros
    df['activity'] = df_1['steps'] + " steps"
    
    df['sleep'] = df_1['sleep_asleep'] + " h" + " (" + df_1['sleep_eff'] + "% eff.)"
    df['sleep'] = df['sleep'].replace('None h (0% eff.)', 'No sleep data.')
    
    return df

In [149]:
def generate_calendar(df, **kwargs): 
    """
    Generates a CSV and ICS from the dataframe
    :param df: cleansed dataframe from `create_description_cols`
    """
    df_event = df[['date', 'food', 'activity', 'sleep']].melt(
        id_vars = ['date'], 
        value_vars = ['food', 'activity', 'sleep'], 
        var_name = 'type', 
        value_name = 'description'
    )
    output_file = kwargs['output_path'] + "/" + kwargs['file_name'] + '.csv'
    df_event.to_csv(output_file)

    c = Calendar()
    for _, row in df_event.iterrows(): 
        e = create_event(row['date'], row['type'], row['description'])
        c.events.add(e)

    with open(kwargs['file_name'] + '.ics', 'w') as f:
        f.write(str(c))
        f.close()

    return df 

In [177]:
def transform(df):
    """
    Round all numerical columns to closest integer except for sleep times and weight
    :param df: dataframe from the read_raw_files function
    """
    if len(df) > 0:
        df = rename_columns(df)
        df = round_df(df)
        df = dedup_df(df)
        df = create_numeric_cols(df)
        df = create_description_cols(df)

        return df

In [178]:
def etl_raw_data(input_path, output_path):
    """
    Perform ETL on Apple Health data
    :param input_path: directory path as type string
    :param output_path:
    """

    df = read_raw_files(input_path)
    df = transform(df)
    df = generate_calendar(df, output_path)

    return

In [161]:
def create_event(date, type, description):
    """
    Create an all day event for the given date and type
    :param date: date as type datetime.date
    :param type: type of event as type string
    :param description: description of event as type string
    """
    if type == 'sleep':
        emoticon = "💤"
    if type == 'activity':
        emoticon = "🔥"
    if type == 'food':
        emoticon = "🥞"

    all_day_date = str(date) + " 00:00:00"
    e = Event()
    e.name = emoticon + " " + description
    e.begin = all_day_date
    e.end = all_day_date
    e.make_all_day()

    return e

In [252]:
import pandas as pd
from datetime import datetime
# %%
def ts_to_dt(ts):
    return datetime.fromtimestamp(ts)

def process_health_data(file):
    """
    Create [date, source] from file.
    :param file: as exported by Auto Health Export
    """
    df = pd.read_csv(file, sep = ',')
    print(f'Processing: {file}')
    df['creation_date'] = ts_to_dt(file.stat().st_atime)
    df['filename'] = file.name

    return df

def read_raw_files(str_path):
    """
    Read all files in a directory and return a dataframe.
    :param str_path: directory path as type string
    """
    df_health = pd.DataFrame()
    df_sleep = pd.DataFrame()
    # valid_files = ['HealthAutoExport', 'AutoSleep']
    print('Reading files..')
    for i in os.scandir(str_path):
        if i.name.endswith('.csv'):
            df_tmp = process_health_data(i)
            if i.name.startswith('HealthAutoExport'):
                df_health = pd.concat([df_health, df_tmp])
            elif i.name.startswith('AutoSleep'):
                df_sleep = pd.concat([df_sleep, df_tmp])

    # concat results in weird indices
    df_health = df_health.reset_index(drop=True)
    return df_health, df_sleep

In [267]:
df_health, df_sleep = read_raw_files('/Users/ntonthat/Library/Mobile Documents/iCloud~com~ifunography~HealthExport/Documents/raw')

Reading files..
Processing: <DirEntry 'HealthAutoExport-2022-06-23-2022-06-29 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-06-2022-07-06 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-07-2022-07-07 Data.csv'>
Processing: <DirEntry 'AutoSleep-20220601-to-20220630.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-06-28-2022-06-28 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-01-2022-07-01 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-06-01-2022-06-27 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-04-2022-07-04 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-06-30-2022-06-30 Data.csv'>
Processing: <DirEntry 'AutoSleep-20220601-to-20220710.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-09-2022-07-09 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-07-03-2022-07-09 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-06-26-2022-06-26 Data.csv'>
Processing: <DirEntry 'HealthAutoExport-2022-06-27-2022-06-27

In [268]:
df_health = transform(df_health)

Adding commas as separator
date : object
carbs : int64
protein : int64
sleep_asleep : float64
sleep_in_bed : float64
steps : int64
fat : int64
weight : float64
creation_date : datetime64[ns]
filename : object
calories : int64
sleep_eff : int64
Creating description columns


In [269]:
df_sleep

,ISO8601,fromDate,toDate,bedtime,waketime,inBed,awake,fellAsleepIn,sessions,asleep,...,SpO2Avg,SpO2Min,SpO2Max,respAvg,respMin,respMax,tags,notes,creation_date,filename
0,2022-06-01T19:59:59+10:00,"Tuesday, 31 May 2022","Wednesday, 1 Jun 2022",2022-05-31 20:47:00,2022-06-01 06:06:00,09:19:00,00:18:00,00:00:00,1,09:01:00,...,95.4,85,100,14.5,12.5,18.5,NaN,NaN,2022-07-10 21:59:40.419715,AutoSleep-20220601-to-20220630.csv
1,2022-06-02T19:59:59+10:00,"Wednesday, 1 Jun 2022","Thursday, 2 Jun 2022",2022-06-01 22:04:00,2022-06-02 05:11:00,07:07:00,00:00:00,00:00:00,1,07:07:00,...,96.3,95,98,13.9,11.0,19.5,NaN,NaN,2022-07-10 21:59:40.419715,AutoSleep-20220601-to-20220630.csv
2,2022-06-03T19:59:59+10:00,"Thursday, 2 Jun 2022","Friday, 3 Jun 2022",2022-06-02 22:03:00,2022-06-03 05:36:00,07:33:00,00:00:00,00:00:00,1,07:33:00,...,96.2,93,99,14.6,12.5,17.5,NaN,NaN,2022-07-10 21:59:40.419715,AutoSleep-20220601-to-20220630.csv
3,2022-06-04T19:59:59+10:00,"Friday, 3 Jun 2022","Saturday, 4 Jun 2022",2022-06-03 21:01:00,2022-06-04 06:35:00,09:34:00,02:03:00,00:00:00,1,07:31:00,...,97.3,96,99,14.0,11.0,24.0,NaN,NaN,2022-07-10 21:59:40.419715,AutoSleep-20220601-to-20220630.csv
4,2022-06-05T19:59:59+10:00,"Saturday, 4 Jun 2022","Sunday, 5 Jun 2022",2022-06-04 22:45:00,2022-06-05 07:29:00,08:44:00,00:00:00,00:00:00,1,08:44:00,...,96.7,95,99,13.7,11.0,20.0,NaN,NaN,2022-07-10 21:59:40.419715,AutoSleep-20220601-to-20220630.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,2022-07-06T19:59:59+10:00,"Tuesday, 5 Jul 2022","Wednesday, 6 Jul 2022",2022-07-04 22:45:00,2022-07-05 06:25:00,07:40:00,00:15:00,00:00:00,1,07:25:00,...,97.2,94,100,14.3,12.5,20.5,NaN,NaN,2022-07-10 21:59:40.458395,AutoSleep-20220601-to-20220710.csv
35,2022-07-07T19:59:59+10:00,"Wednesday, 6 Jul 2022","Thursday, 7 Jul 2022",2022-07-05 23:17:00,2022-07-06 07:31:00,08:14:00,00:00:00,00:00:00,1,08:14:00,...,96.5,94,99,13.8,12.0,16.5,NaN,NaN,2022-07-10 21:59:40.458395,AutoSleep-20220601-to-20220710.csv
36,2022-07-08T19:59:59+10:00,"Thursday, 7 Jul 2022","Friday, 8 Jul 2022",2022-07-06 22:29:00,2022-07-07 06:17:00,07:48:00,00:00:00,00:00:00,1,07:48:00,...,95.7,94,97,13.4,10.5,17.0,NaN,NaN,2022-07-10 21:59:40.458395,AutoSleep-20220601-to-20220710.csv
37,2022-07-09T19:59:59+10:00,"Friday, 8 Jul 2022","Saturday, 9 Jul 2022",2022-07-07 23:16:00,2022-07-08 07:00:00,07:44:00,00:00:00,00:00:00,1,07:44:00,...,96.6,95,99,13.2,10.0,17.0,NaN,NaN,2022-07-10 21:59:40.458395,AutoSleep-20220601-to-20220710.csv


In [271]:
df_health_sleep = etl_autosleep_data(df_sleep)[['date', 'sleep']]

In [272]:
df_health_sleep

,date,sleep
0,2022-06-01,7 h 16 m [2 h 0 m / 96%] \n (🌒 10:30 PM /🌞 6:0...
1,2022-06-02,9 h 1 m [0 h 15 m / 96%] \n (🌒 8:47 PM /🌞 6:06...
2,2022-06-03,7 h 7 m [2 h 17 m / 100%] \n (🌒 10:04 PM /🌞 5:...
3,2022-06-04,7 h 33 m [1 h 30 m / 100%] \n (🌒 10:03 PM /🌞 5...
4,2022-06-05,7 h 31 m [1 h 33 m / 78%] \n (🌒 9:01 PM /🌞 6:3...
5,2022-06-06,8 h 44 m [3 h 14 m / 100%] \n (🌒 10:45 PM /🌞 7...
6,2022-06-07,9 h 34 m [3 h 52 m / 94%] \n (🌒 9:31 PM /🌞 7:3...
7,2022-06-08,8 h 18 m [3 h 16 m / 100%] \n (🌒 9:54 PM /🌞 6:...
8,2022-06-09,8 h 9 m [2 h 53 m / 88%] \n (🌒 10:11 PM /🌞 7:2...
9,2022-06-10,6 h 17 m [2 h 0 m / 90%] \n (🌒 11:05 PM /🌞 6:0...


In [275]:
df_merge = pd.merge(df_health, df_health_sleep,  on = 'date', how= 'left')[['date', 'food', 'activity', 'sleep_x', 'sleep_y']]

In [279]:
df_merge[['date', 'food', 'activity', 'sleep']]

,date,food,activity,sleep
0,2022-06-01,"2,081 calories (279C/104P/61F)","10,957 steps",7 h 16 m [2 h 0 m / 96%] \n (🌒 10:30 PM /🌞 6:0...
1,2022-06-02,"2,293 calories (308C/146P/53F)","10,639 steps",9 h 1 m [0 h 15 m / 96%] \n (🌒 8:47 PM /🌞 6:06...
2,2022-06-03,"2,108 calories (282C/128P/52F)","13,124 steps",7 h 7 m [2 h 17 m / 100%] \n (🌒 10:04 PM /🌞 5:...
3,2022-06-04,"2,658 calories (289C/182P/86F)","9,345 steps",7 h 33 m [1 h 30 m / 100%] \n (🌒 10:03 PM /🌞 5...
4,2022-06-05,"2,334 calories (260C/148P/78F)","8,875 steps",7 h 31 m [1 h 33 m / 78%] \n (🌒 9:01 PM /🌞 6:3...
5,2022-06-06,"2,468 calories (288C/185P/64F)","11,716 steps",8 h 44 m [3 h 14 m / 100%] \n (🌒 10:45 PM /🌞 7...
6,2022-06-07,"2,337 calories (249C/162P/77F)","14,449 steps",9 h 34 m [3 h 52 m / 94%] \n (🌒 9:31 PM /🌞 7:3...
7,2022-06-08,"2,336 calories (284C/147P/68F)","14,780 steps",8 h 18 m [3 h 16 m / 100%] \n (🌒 9:54 PM /🌞 6:...
8,2022-06-09,"2,301 calories (326C/148P/45F)","13,597 steps",8 h 9 m [2 h 53 m / 88%] \n (🌒 10:11 PM /🌞 7:2...
9,2022-06-10,"2,475 calories (305C/154P/71F)","8,614 steps",6 h 17 m [2 h 0 m / 90%] \n (🌒 11:05 PM /🌞 6:0...


In [ ]:
df_health_sleep

,date,sleep
0,2022-06-01,9 h 1 m [0 h 15 m / 96%] \n (🌒 8:47 PM /🌞 6:06...
1,2022-06-02,7 h 7 m [2 h 17 m / 100%] \n (🌒 10:04 PM /🌞 5:...
2,2022-06-03,7 h 33 m [1 h 30 m / 100%] \n (🌒 10:03 PM /🌞 5...
3,2022-06-04,7 h 31 m [1 h 33 m / 78%] \n (🌒 9:01 PM /🌞 6:3...
4,2022-06-05,8 h 44 m [3 h 14 m / 100%] \n (🌒 10:45 PM /🌞 7...
5,2022-06-06,9 h 34 m [3 h 52 m / 94%] \n (🌒 9:31 PM /🌞 7:3...
6,2022-06-07,8 h 18 m [3 h 16 m / 100%] \n (🌒 9:54 PM /🌞 6:...
7,2022-06-08,8 h 9 m [2 h 53 m / 88%] \n (🌒 10:11 PM /🌞 7:2...
8,2022-06-09,6 h 17 m [2 h 0 m / 90%] \n (🌒 11:05 PM /🌞 6:0...
9,2022-06-10,8 h 10 m [1 h 32 m / 100%] \n (🌒 10:14 PM /🌞 5...


In [174]:
def rename_columns(df):
    """
    Rename columns for easier reference
    Styling follows lowercase and no units with spaces being replaced by _
    """
    d_col_rename = {
        'Date': 'date',
        'Carbohydrates (g)': 'carbs',
        'Protein (g)': 'protein',
        'Total Fat (g)': 'fat',
        'Sleep Analysis [In Bed] (hr)': 'sleep_in_bed',
        'Sleep Analysis [Asleep] (hr)': 'sleep_asleep',
        'Step Count (count)': 'steps',
        'Weight & Body Mass (kg) ': 'weight'
    }

    df.rename(columns=d_col_rename, inplace=True)

    # fill in values
    df = df.replace(r'^\s+$', np.nan, regex=True)

    # convert column types
    df = convert_column_types(df)
    return df


def convert_column_types(df):
    """
    Convert certain columns to be a certain type
    """
    df['date'] = pd.to_datetime(df['date']).dt.date

    # force apply float64 type for weight
    df['weight'] = df['weight'].astype(float)

    return df

def dedup_df(df):
    """
    Remove duplicates ordering by 'date' and 'creation_date' and then keep only the latest
    """
    df_sort = df.sort_values(['date', 'creation_date'], ascending= True)
    df_dedup = df_sort.drop_duplicates(subset = 'date', keep = 'last')

    return df_dedup

# %%
def create_description_cols(df):
    """
    Create description columns for the generating events
    """
    print("Adding commas as separator")
    for i in df.columns:
        print(f"{i} : {df[i].dtypes}")
        if df[i].dtypes == 'float64':
            df[i] = df[i].apply(lambda x: f"{x:,.1f}")
        elif df[i].dtypes == 'int64':
            df[i] = df[i].apply(lambda x: f"{x:,.0f}")

    print("Creating description columns")
    df_1 = df.astype(str)

    food_macros = "(" + df_1['carbs'] + "C/" + df_1['protein'] + "P/" + df_1['fat'] + "F" + ")"
    df['food'] = df_1['calories'] + " calories " + food_macros
    df['activity'] = df_1['steps'] + " steps"

    df['sleep'] = df_1['sleep_asleep'] + " h" + " (" + df_1['sleep_eff'] + "% eff.)"
    df['sleep'] = df['sleep'].replace('nan h (0% eff.)', 'No sleep data.')

    return df

def create_numeric_cols(df):
    """
    Calculates the total calories from the macros
    Calculates the sleep efficiency
    """
    df['calories'] = df['carbs'] * 4 + df['fat'] * 9 + df['protein'] * 4
    df['sleep_eff'] = df['sleep_asleep'] / df['sleep_in_bed'] * 100
    df['sleep_eff'] = df['sleep_eff'].fillna(0)
    df['sleep_eff'] = df['sleep_eff'].astype(int)

    return df

# %%
def round_df(df):
    """
    Round all numerical columns to closest integer except for one d.p. cols
    Replaces all NaN with null
    """
    one_dp_cols = ['sleep_asleep', 'sleep_in_bed', 'weight']
    for i in df.columns:
        if df[i].dtypes == 'float64':
            if i in one_dp_cols:
                df[i] = df[i].round(1)
            else:
                df[i] = df[i].astype(int)

    # df = df.replace({np.nan: None})
    return df

Adding commas as separator
date : object
carbs : int64
protein : int64
sleep_asleep : float64
sleep_in_bed : float64
steps : int64
fat : int64
weight : float64
creation_date : datetime64[ns]
filename : object
calories : int64
sleep_eff : int64
Creating description columns


In [180]:
df_health

,date,carbs,protein,sleep_asleep,sleep_in_bed,steps,fat,weight,creation_date,filename,calories,sleep_eff,food,activity,sleep
11,2022-06-01,279,104,9.0,9.3,"10,957",61,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,081",96,"2,081 calories (279C/104P/61F)","10,957 steps",9.0 h (96% eff.)
12,2022-06-02,308,146,nan,nan,"10,639",53,72.4,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,293",0,"2,293 calories (308C/146P/53F)","10,639 steps",No sleep data.
13,2022-06-03,282,128,7.6,7.6,"13,124",52,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,108",100,"2,108 calories (282C/128P/52F)","13,124 steps",7.6 h (100% eff.)
14,2022-06-04,289,182,8.7,8.7,"9,345",86,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,658",100,"2,658 calories (289C/182P/86F)","9,345 steps",8.7 h (100% eff.)
15,2022-06-05,260,148,nan,nan,"8,875",78,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,334",0,"2,334 calories (260C/148P/78F)","8,875 steps",No sleep data.
16,2022-06-06,288,185,9.6,10.1,"11,716",64,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,468",95,"2,468 calories (288C/185P/64F)","11,716 steps",9.6 h (95% eff.)
17,2022-06-07,249,162,nan,nan,"14,449",77,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,337",0,"2,337 calories (249C/162P/77F)","14,449 steps",No sleep data.
18,2022-06-08,284,147,8.2,9.2,"14,780",68,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,336",89,"2,336 calories (284C/147P/68F)","14,780 steps",8.2 h (89% eff.)
19,2022-06-09,326,148,6.3,7.0,"13,597",45,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,301",90,"2,301 calories (326C/148P/45F)","13,597 steps",6.3 h (90% eff.)
20,2022-06-10,305,154,8.2,9.4,"8,614",71,nan,2022-07-10 21:41:38.043512,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,"2,475",87,"2,475 calories (305C/154P/71F)","8,614 steps",8.2 h (87% eff.)
